In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline 

In [3]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from common import rotate_to_sector0,uv2sectoru0v0

In [4]:
## First preliminary geometry file
df_geom_bestguess = pd.read_csv('../data/geometry_bestguess_v2.hgcal.txt', sep=' ')
## Geometry file as of Feb. 16 2023 (https://gitlab.cern.ch/hgcal-integration/hgcal_modmap/-/blob/development/geometries/v15.2/geometry.hgcal.txt)
df_geom_v15p2 = pd.read_csv('../data/geometry.hgcal.v15.2.txt', sep=' ')
df_constr = pd.read_csv('../data/220712_strobbe_elinks-mapping.csv')
showermax_layers = [7,9,11,13]
hcal_start = 27
elinks_showermax = 4
elinks_regular = 2
# STC: assuming STC4 5E+4M in HDM and STC16 in LDM
elinks_stc16 = 2
elinks_stc4 = 5

In [5]:
df_constr = df_constr.set_index(['layer', 'u', 'v'])

In [6]:
def get_constr_bcstc(row):
    layer = int(row.plane)
    u = int(row.u)
    v = int(row.v)
    # Rotate to 60deg sector0 if in CE-E
    u0 = int(row.u0) if layer<hcal_start else u
    v0 = int(row.v0) if layer<hcal_start else v
    itype = row.itype
    triangle = itype[0]=='c'
    hdm = 'I' in itype
    bcstc_elinks = 0
    constr_elinks = 999
    bcstc_elinks = elinks_showermax if layer in showermax_layers else elinks_regular
    # no trigger links in even layers
    if layer%2==0 and layer<hcal_start: bcstc_elinks = 0 
    # STC elinks: STC4 in HDM and STC16 in LDM
    if layer>=hcal_start:
        if hdm:
            bcstc_elinks = elinks_stc4
        else:
            bcstc_elinks = elinks_stc16
    layer_tmp = min(layer, hcal_start) # all HCAL layers have the same constraints
    if df_constr.index.isin([(layer_tmp, u0, v0)]).any():
        constr_elinks = int(df_constr.loc[(layer_tmp,u0,v0)])
    # No trigger elinks in triangles/three-partials
    if triangle: constr_elinks = 0
    return min(bcstc_elinks, constr_elinks)

In [7]:
def process_geom(df_geom):
    df_geom = df_geom[['plane','u','v','trigLinks','itype']]
    # Get sector 0 u and v
    df_geom[['sector60', 'u0', 'v0']] = df_geom.apply(uv2sectoru0v0, axis=1)
    # get constrained number of elinks for BC+STC
    df_geom['trigLinksBCSTC'] = df_geom.apply(get_constr_bcstc, axis=1)
    # Compare the number of trigger elinks in the geometry to the constrained number
    # There is an issue if we have less than the constrained one
    df_geom['ok'] = df_geom['trigLinks']>=df_geom['trigLinksBCSTC']
    return df_geom
    

In [9]:
df_geom_bestguess = process_geom(df_geom_bestguess)
df_geom_v15p2 = process_geom(df_geom_v15p2)

/home/llr/cms/sauvan/.local/lib/python3.7/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [10]:
df_geom_bestguess.query('ok==False')

,plane,u,v,trigLinks,itype,sector60,u0,v0,trigLinksBCSTC,ok


In [11]:
df_geom_v15p2.query('ok==False')

,plane,u,v,trigLinks,itype,sector60,u0,v0,trigLinksBCSTC,ok


In [12]:
df_geom_bestguess = df_geom_bestguess.reset_index().set_index(['plane','u','v'])
df_geom_v15p2 = df_geom_v15p2.reset_index().set_index(['plane','u','v'])

In [43]:
## Compte the differences in number of trigger elinks between the two geometry files
df_diff = df_geom_v15p2['trigLinks'] - df_geom_bestguess['trigLinks']
## Extract NaNs, meaning wafer not existing in one of the files 
df_diff_nans = df_geom_bestguess[df_diff.isna()]
## Extract differences fo
df_diff = df_diff.dropna()
df_diff[df_diff!=0]

plane  u   v 
27     1   10    2.0
       10  9     2.0
32     1   11    2.0
       11  10    2.0
Name: trigLinks, dtype: float64

In [45]:
df_diff_nans[df_diff_nans.trigLinks>0]

trigLinks itype  sector60  u0  v0  trigLinksBCSTC    ok
plane u  v                                                          
27    9  0         2.0  aOeB         0   9   0               2  True
         9         2.0  aOeB         1   9   0               2  True
28    9  0         2.0  aOeB         0   9   0               2  True
         9         2.0  aOeB         1   9   0               2  True
29    9  0         2.0  aOeB         0   9   0               2  True
         9         2.0  aOeB         1   9   0               2  True
30    9  0         2.0  aOeB         0   9   0               2  True
         9         2.0  aOeT         1   9   0               2  True
31    9  0         2.0  aOeB         0   9   0               2  True
         9         2.0  aOeB         1   9   0               2  True
32    10 0         2.0  aOeB         0  10   0               2  True
         10        2.0  aOeB         1  10   0               2  True
33    10 0         2.0  aOeB         0  10   0               2  True
         10        2.0  aOeB         1  10   0               2  True